https://github.com/hwchase17/langchain/blob/master/docs/modules/chains/examples/multi_retrieval_qa_router.ipynb

In [2]:
from langchain.chains.router import MultiRetrievalQAChain
from langchain.llms import OpenAI

import util_funs.globalsettings as gs

In [3]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma

vector_store_pdf = Chroma(
    collection_name='pdf_files',
    embedding_function=OpenAIEmbeddings(),
    persist_directory=gs.the_folders.DIR_PERSIST
    )
pdf_retriever = vector_store_pdf.as_retriever()

vector_store_csv = Chroma(
    collection_name='csv_files',
    embedding_function=OpenAIEmbeddings(),
    persist_directory=gs.the_folders.DIR_PERSIST
    )
csv_retriever = vector_store_pdf.as_retriever()

Using embedded DuckDB with persistence: data will be stored in: C:\Users\migue\Documents\PROYECTOS DATA SCIENCE\langchain\PYTHON\util_funs\..\..\data\chroma
Using embedded DuckDB with persistence: data will be stored in: C:\Users\migue\Documents\PROYECTOS DATA SCIENCE\langchain\PYTHON\util_funs\..\..\data\chroma


In [42]:
vector_store_csv.

VectorStoreRetriever(vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x000001BDFFEFF8D0>, search_type='similarity', search_kwargs={})

In [34]:
retriever_infos = [
    {
        "name": "situacion economia", 
        "description": "Good for answering questions about economy", 
        "retriever": pdf_retriever
    },
    {
        "name": "datos paro", 
        "description": "Good for answering questions about unemployement", 
        "retriever": csv_retriever
    }
]

In [17]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [35]:
chain = MultiRetrievalQAChain.from_retrievers(OpenAI(), retriever_infos, verbose=True)

In [36]:
print(chain.run("Qué impacto tuvo la guerra de Ucrania en la economia durante 2022?"))



> Entering new MultiRetrievalQAChain chain...
situacion economia: {'query': '¿Cuál fue el impacto de la guerra de Ucrania en la economía durante 2022?'}
> Finished chain.
 La guerra de Ucrania afectó a la economía española durante 2022 tanto en los precios como en la actividad, a través de varios canales, entre los que destaca el alza de los costes energéticos que se tradujo en unos niveles de inflación elevados que retraen el consumo. El incremento de los precios está siendo más acusado y está durando más de lo inicialmente previsto.


In [43]:
print(chain.run("cual es la tasa de paro total nacional en españa del 2023T1?"))



> Entering new MultiRetrievalQAChain chain...
datos paro: {'query': 'cual es la tasa de paro total nacional en españa del 2023T1?'}
> Finished chain.
 La tasa de paro total nacional en España para el primer trimestre de 2023 se prevé que sea del 12%.
